# Taller 1: Limpieza y Análisis de Datos de Reaseguro

## 1. Carga de Datos

En esta sección, cargamos los datos desde el archivo Excel. Cada hoja del archivo se carga en un DataFrame de pandas independiente.

In [52]:
# Importar la librería pandas para el manejo de datos
import pandas as pd

# Definir la ruta del archivo Excel que contiene los datos
excel_path = 'data/QS - Data ejercicios_sin_modificar.xlsx'

# Leer todas las hojas del archivo Excel y almacenarlas en un diccionario de DataFrames
data = pd.read_excel(excel_path, sheet_name=None)

# Iterar sobre el diccionario de DataFrames para crear una variable global por cada hoja
# Esto permite acceder a cada DataFrame por el nombre de su hoja (ej. Siniestros, Primas, etc.)
for sheet, df in data.items():
    # Reemplazar espacios en los nombres de las hojas con guiones bajos para nombres de variables válidos
    variable_name = sheet.replace(' ', '_')
    globals()[variable_name] = df
    print(f"Se creó el DataFrame: '{variable_name}'")

Se creó el DataFrame: 'Primas'
Se creó el DataFrame: 'Siniestros'
Se creó el DataFrame: 'Avisados'
Se creó el DataFrame: 'Reaseguro'


## 2. Limpieza y Preparación del DataFrame `Siniestros`

In [53]:
# Ordenar el DataFrame 'Siniestros' por la columna 'SINIESTRO' de forma ascendente
Siniestros.sort_values(by="SINIESTRO", ascending=True, inplace=True)
# Crear una columna 'ID' que sirva como identificador único para cada siniestro
# Se basa en el índice del DataFrame, asegurando un número consecutivo para cada registro
Siniestros['ID'] = Siniestros.index + 1

### 2.1. Ordenamiento y Creación de ID Único

Para facilitar el seguimiento de los registros, primero ordenamos el DataFrame por el número de siniestro y luego creamos una columna `ID` con un identificador único para cada fila.

In [54]:
# Convertir la columna 'FECHA SINIESTRO' al formato datetime
# Se crea una nueva columna 'Fecha de Ocurrencia' para almacenar los valores convertidos
Siniestros['Fecha de Ocurrencia'] = pd.to_datetime(Siniestros['FECHA SINIESTRO'])

C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\464805755.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Siniestros['Fecha de Ocurrencia'] = pd.to_datetime(Siniestros['FECHA SINIESTRO'])


### 2.2. Limpieza y Conversión de Fechas

Las columnas de fechas son cruciales para el análisis. En esta sección, convertimos las columnas de fecha a un formato estándar (`datetime`) y manejamos los errores que puedan surgir durante la conversión.

In [55]:
# Convertir la columna 'FECHA AVISO' a datetime. 'coerce' convierte los errores en NaT (Not a Time)
Siniestros['Fecha de Pago'] = pd.to_datetime(Siniestros['FECHA AVISO'], errors='coerce')

# --- Corrección de errores específicos: Espacios en blanco ---
# Identificar las filas donde 'FECHA AVISO' es un espacio en blanco
errores_espacio = Siniestros[Siniestros['FECHA AVISO'] == " "]
id_errores_espacio = errores_espacio['ID'].tolist()

# Para estos casos, se asume que la fecha de aviso es la misma que la fecha de ocurrencia
Siniestros.loc[Siniestros['ID'].isin(id_errores_espacio), 'Fecha de Pago'] = Siniestros.loc[Siniestros['ID'].isin(id_errores_espacio), 'Fecha de Ocurrencia']

# Verificar que ya no queden errores de espacios en blanco
errores_espacio2 = Siniestros[Siniestros['FECHA AVISO'] == " "]
print(f"Número de errores de espacio restantes: {len(errores_espacio2)}")

Número de errores de espacio restantes: 8


C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\2642177191.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Siniestros['Fecha de Pago'] = pd.to_datetime(Siniestros['FECHA AVISO'], errors='coerce')


In [56]:
# --- Corrección de errores específicos: Formatos numéricos (ej. 201901) ---
# Identificar las filas donde la conversión a fecha falló (resultando en NaT)
errores_fecha_aviso = Siniestros[Siniestros['Fecha de Pago'].isnull() & Siniestros['FECHA AVISO'].notnull()]
print(f"Se encontraron {len(errores_fecha_aviso)} registros con formato de fecha numérico.")

Se encontraron 1096 registros con formato de fecha numérico.


In [57]:
# Convertir la columna 'FECHA AVISO' de los errores a string para poder manipularla
errores_fecha_aviso['FECHA AVISO'] = errores_fecha_aviso['FECHA AVISO'].astype(str)

# Extraer el año (primeros 4 dígitos) y el mes (dígitos restantes)
errores_fecha_aviso['año_aviso'] = errores_fecha_aviso['FECHA AVISO'].str[:4]
errores_fecha_aviso['mes_aviso'] = errores_fecha_aviso['FECHA AVISO'].str[4:]

# Reconstruir la fecha en formato 'YYYY/MM/DD', asumiendo el día 01
errores_fecha_aviso['Fecha de Pago'] = errores_fecha_aviso['año_aviso'] + "/" + errores_fecha_aviso['mes_aviso'] + "/01"

# Convertir la fecha reconstruida al formato datetime
errores_fecha_aviso['Fecha de Pago'] = pd.to_datetime(errores_fecha_aviso['Fecha de Pago'], format='%Y/%m/%d', errors='coerce')

# --- Actualización del DataFrame Principal ---
# Crear un mapa de ID a 'Fecha de Pago' corregida desde el DataFrame de errores
id_a_fecha_mapa = errores_fecha_aviso.set_index('ID')['Fecha de Pago']

# Actualizar el DataFrame 'Siniestros' usando el mapa.
# .map() aplica la corrección y .fillna() mantiene los valores originales que no necesitaban corrección.
Siniestros['Fecha de Pago'] = Siniestros['ID'].map(id_a_fecha_mapa).fillna(Siniestros['Fecha de Pago'])
print("El DataFrame 'Siniestros' ha sido actualizado con las fechas corregidas.")

El DataFrame 'Siniestros' ha sido actualizado con las fechas corregidas.


C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\827445970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errores_fecha_aviso['FECHA AVISO'] = errores_fecha_aviso['FECHA AVISO'].astype(str)
C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\827445970.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errores_fecha_aviso['año_aviso'] = errores_fecha_aviso['FECHA AVISO'].str[:4]
C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\827445970.py:6: SettingWithCopyWarning: 
A value is trying

In [58]:
# Calcular la diferencia en días entre la fecha de aviso y la fecha de ocurrencia
# El resultado se almacena en una nueva columna llamada 'dias_entre_fechas'
Siniestros['dias_entre_fechas'] = (Siniestros['Fecha de Pago'] - Siniestros['Fecha de Ocurrencia']).dt.days
print("Se ha calculado la diferencia de días entre la fecha de aviso y ocurrencia.")

Se ha calculado la diferencia de días entre la fecha de aviso y ocurrencia.


### 2.3. Análisis de Consistencia de Fechas

Una vez limpias las fechas, es importante verificar su consistencia. Un indicador clave es la diferencia de días entre la fecha de ocurrencia del siniestro y la fecha de aviso. Lógicamente, la fecha de aviso no debería ser anterior a la de ocurrencia.

In [59]:
# Contar el número de registros donde la diferencia de días es negativa (inconsistencia)
num_datos_negativos = Siniestros[Siniestros['dias_entre_fechas'] < 0].shape[0]
print(f"Número de registros con fecha de aviso anterior a la de ocurrencia: {num_datos_negativos}")

# Calcular el porcentaje de datos inconsistentes sobre el total
num_datos_totales = Siniestros.shape[0]
porcentaje_negativos = (num_datos_negativos / num_datos_totales) * 100
print(f"Porcentaje de datos inconsistentes: {porcentaje_negativos:.2f}%")

# Si el porcentaje de inconsistencias es bajo (menor al 3%), se procede a eliminarlos
if porcentaje_negativos < 3:
    Siniestros = Siniestros[Siniestros['dias_entre_fechas'] >= 0]
    print(f"Se han eliminado {num_datos_negativos} registros inconsistentes.")
else:
    print("El porcentaje de datos inconsistentes es alto. Se recomienda una revisión manual.")

Número de registros con fecha de aviso anterior a la de ocurrencia: 532
Porcentaje de datos inconsistentes: 1.13%
Se han eliminado 532 registros inconsistentes.


## 3. Limpieza y Preparación del DataFrame `Avisados`

In [60]:
# Convertir 'FECHA SINIESTRO' a datetime, creando la columna 'Fecha de Ocurrencia'
Avisados['Fecha de Ocurrencia'] = pd.to_datetime(Avisados['FECHA SINIESTRO'], errors='coerce')

# Verificar si ocurrieron errores durante la conversión
if Avisados['Fecha de Ocurrencia'].isnull().any():
    print("Se encontraron errores en la conversión de 'FECHA SINIESTRO' en el DataFrame Avisados.")
else:
    print("La columna 'FECHA SINIESTRO' de Avisados se convirtió correctamente.")

La columna 'FECHA SINIESTRO' de Avisados se convirtió correctamente.


### 3.1. Limpieza y Conversión de Fechas

Al igual que con `Siniestros`, el primer paso es convertir las columnas de fecha a un formato estándar y manejar cualquier error.

In [61]:
# Convertir 'FECHA AVISO' a datetime, creando la columna 'Fecha de Aviso'
Avisados["Fecha de Aviso"]= pd.to_datetime(Avisados["FECHA AVISO"], errors='coerce')

# Verificar si ocurrieron errores durante la conversión
if Avisados["Fecha de Aviso"].isnull().any():
    print("Se encontraron errores en la conversión de 'FECHA AVISO' en el DataFrame Avisados.")
else:
    print("La columna 'FECHA AVISO' de Avisados se convirtió correctamente.")

Se encontraron errores en la conversión de 'FECHA AVISO' en el DataFrame Avisados.


In [62]:
# Identificar y almacenar las filas donde la conversión de 'Fecha de Aviso' falló
errores_fecha_aviso_avisados = Avisados[Avisados['Fecha de Aviso'].isnull()]
print(f"Se encontraron {len(errores_fecha_aviso_avisados)} filas con errores en 'Fecha de Aviso' en el DataFrame Avisados.")

Se encontraron 6 filas con errores en 'Fecha de Aviso' en el DataFrame Avisados.


In [63]:
# Eliminar las filas que contienen errores en la 'Fecha de Aviso'
Avisados = Avisados.dropna(subset=['Fecha de Aviso'])
print(f"Se eliminaron las filas con errores. El DataFrame Avisados ahora tiene {len(Avisados)} filas.")

Se eliminaron las filas con errores. El DataFrame Avisados ahora tiene 835 filas.


In [64]:
# Calcular la diferencia en días entre 'Fecha de Aviso' y 'Fecha de Ocurrencia'
Avisados['Dias Entre'] = (Avisados['Fecha de Aviso'] - Avisados['Fecha de Ocurrencia']).dt.days
print("Se ha calculado la diferencia de días para el DataFrame Avisados.")

Se ha calculado la diferencia de días para el DataFrame Avisados.


C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\2125196292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Avisados['Dias Entre'] = (Avisados['Fecha de Aviso'] - Avisados['Fecha de Ocurrencia']).dt.days


### 3.2. Análisis de Consistencia de Fechas

Similar al DataFrame `Siniestros`, verificamos la consistencia entre la fecha de ocurrencia y la fecha de aviso.

In [65]:
#si el numero de dias entre la fecha de aviso y la fecha de ocurrencia es negativo, entonces hacer la fecha de ocurrencia igual a la de aviso
Avisados.loc[Avisados['Dias Entre'] < 0, 'Fecha de Ocurrencia'] = Avisados['Fecha de Aviso']

In [66]:
#hacemos los dias entre fecha de aviso y fecha de ocurrencia recalculo
Avisados['Dias Entre'] = (Avisados['Fecha de Aviso'] - Avisados['Fecha de Ocurrencia']).dt.days

C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\2995833068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Avisados['Dias Entre'] = (Avisados['Fecha de Aviso'] - Avisados['Fecha de Ocurrencia']).dt.days


# Arreglar los periodos de desarrollo


In [67]:
#para la base de siniestros crear una columna que coloque el año anterior de la fecha de ocurrencia si la fecha de ocurrencia es anterior al 1 de julio de ese año de lo contrario colocar el mismo año

Siniestros['Vigencia de Ocurrencia'] = Siniestros['Fecha de Ocurrencia'].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year, month=7, day=1) else x.year)
Siniestros['Vigencia de Pago'] = Siniestros["Fecha de Pago"].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year, month=7, day=1) else x.year)

C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\3322055405.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Siniestros['Vigencia de Ocurrencia'] = Siniestros['Fecha de Ocurrencia'].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year, month=7, day=1) else x.year)
C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\3322055405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Siniestros['Vigencia de Pago'] = Siniestros["Fecha de Pago"].apply(lambda x: x.year - 1 if x < pd.Timestamp(year

La columna `VIGENCIA` se calcula de la siguiente manera:
- Si la `Fecha de Ocurrencia` es anterior al 1 de julio del año en curso, la `VIGENCIA` será el año anterior.
- De lo contrario, la `VIGENCIA` será el mismo año de la `Fecha de Ocurrencia`.

In [68]:
Avisados['Vigencia de Ocurrencia'] = Avisados['Fecha de Ocurrencia'].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year, month=7, day=1) else x.year)
Avisados['Vigencia de Aviso'] = Avisados["Fecha de Aviso"].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year, month=7, day=1) else x.year)

C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\708453331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Avisados['Vigencia de Ocurrencia'] = Avisados['Fecha de Ocurrencia'].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year, month=7, day=1) else x.year)
C:\Users\sergio_math\AppData\Local\Temp\ipykernel_23664\708453331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Avisados['Vigencia de Aviso'] = Avisados["Fecha de Aviso"].apply(lambda x: x.year - 1 if x < pd.Timestamp(year=x.year,

In [69]:
#exportar para excel los dataframes en cada hoja del excel

# Crear un objeto ExcelWriter
with pd.ExcelWriter('QS_limpio.xlsx') as writer:
    Avisados.to_excel(writer, sheet_name='Avisados',index=False)
    Siniestros.to_excel(writer, sheet_name='Siniestros',index=False)
    Primas.to_excel(writer, sheet_name='Primas',index=False)
    Reaseguro.to_excel(writer, sheet_name='Reaseguro',index=False)
